In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Concatenate
from keras.callbacks import EarlyStopping

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("combined_final.csv")
ev_df = pd.read_csv(file_path)
from pathlib import Path

# Review the DataFrame
ev_df.head()

,City,State,Total EV Chargers,Median Income (16 and over),Total Population
0,Abilene,Texas,23,35115,128149
1,Akron,Ohio,80,36969,188498
2,Alameda,California,145,82773,76039
3,Albany,Georgia,554,26766,67922
4,Albany,New York,554,26766,100832


In [3]:
# Defining a threshold for the number of chargers to determine EV station presence
charger_threshold = 10

In [4]:
# Creating a new column 'EV_Station_Presence' based on the threshold
ev_df['EV_Station_Presence'] = ev_df['Total EV Chargers'].apply(lambda x: 1 if x >= charger_threshold else 0)

In [5]:
# Feature columns used for prediction (including City)
feature_columns = ['Total EV Chargers', 'Median Income (16 and over)', 'Total Population', 'City']

In [6]:
# Creating feature matrix X and target vector y
X = ev_df[feature_columns]
y = ev_df['EV_Station_Presence']

In [7]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Preprocessing numerical features (scaling)
num_columns = ['Total EV Chargers', 'Median Income (16 and over)', 'Total Population']
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[num_columns])
X_test_num = scaler.transform(X_test[num_columns])

In [9]:
# Preprocessing categorical feature (one-hot encoding)
cat_column = ['City']
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_cat = encoder.fit_transform(X_train[cat_column])
X_test_cat = encoder.transform(X_test[cat_column])

D:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
# Concatenating numerical and categorical features
X_train_final = Concatenate()([X_train_num, X_train_cat])
X_test_final = Concatenate()([X_test_num, X_test_cat])

In [11]:
# Building the neural network model with modifications as discussed
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=X_train_final.shape[1]))  # Example reduction in model complexity
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Training the model with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train_final, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_final, y_test), callbacks=[early_stop])

Epoch 1/100
16/16 [==============================] - 1s 11ms/step - loss: 0.7010 - accuracy: 0.4032 - val_loss: 0.6648 - val_accuracy: 0.8960
Epoch 2/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6532 - accuracy: 0.8548 - val_loss: 0.6234 - val_accuracy: 0.9280
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6060 - accuracy: 0.9637 - val_loss: 0.5874 - val_accuracy: 0.9280
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5611 - accuracy: 0.9657 - val_loss: 0.5519 - val_accuracy: 0.9280
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5174 - accuracy: 0.9657 - val_loss: 0.5147 - val_accuracy: 0.9280
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4738 - accuracy: 0.9657 - val_loss: 0.4774 - val_accuracy: 0.9280
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4275 - accuracy: 0.9657 - val_loss: 0.4408 - val_accuracy: 0.9280
Epoch 8/100


Epoch 59/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0263 - accuracy: 0.9940 - val_loss: 0.2034 - val_accuracy: 0.9280
Epoch 60/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0238 - accuracy: 0.9980 - val_loss: 0.2032 - val_accuracy: 0.9280
Epoch 61/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0215 - accuracy: 1.0000 - val_loss: 0.2014 - val_accuracy: 0.9280
Epoch 62/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0207 - accuracy: 0.9980 - val_loss: 0.2006 - val_accuracy: 0.9280
Epoch 63/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0202 - accuracy: 1.0000 - val_loss: 0.2004 - val_accuracy: 0.9280
Epoch 64/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 0.2002 - val_accuracy: 0.9280
Epoch 65/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 0.1991 - val_accuracy: 0.9280
Epoch 

In [13]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_final, y_test)
print(f"Neural Network Accuracy: {accuracy}")

# Calculating validation metrics
y_pred = model.predict(X_test_final)
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert predicted probabilities to classes

precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

4/4 [==============================] - 0s 2ms/step - loss: 0.1930 - accuracy: 0.9280
Neural Network Accuracy: 0.9279999732971191
4/4 [==============================] - 0s 1ms/step
Precision: 0.928
Recall: 1.0
F1-score: 0.9626556016597512
